<a href="https://colab.research.google.com/github/dhsy3206/Leaked-GPTs/blob/main/%EA%B4%80%EA%B4%91%EC%A7%80%EC%8B%9C%EA%B0%81%ED%99%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
!pip install pandas folium



In [78]:
# 필요한 라이브러리 임포트
import pandas as pd
import folium
from google.colab import files

# 파일 업로드
uploaded = files.upload()

# 업로드된 엑셀 파일 로드
df = pd.read_excel(next(iter(uploaded)))

# 데이터프레임 정보 출력
print(df.head())
print(df.info())

# folium 지도 생성 (지도 중심은 데이터프레임의 위도와 경도의 평균값)
center_lat, center_lon = df['위도'].mean(), df['경도'].mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# 관광지 마커 추가
for idx, row in df.iterrows():
    folium.Marker([row['위도'], row['경도']], popup=row['관광지명']).add_to(m)

# 지도 출력
m.save('map.html')
m


Saving tourist_spots.xlsx to tourist_spots (10).xlsx
       관광지명       위도        경도
0       경복궁  37.5789  126.9768
1      남산타워  37.5514  126.9880
2   해운대해수욕장  35.1587  129.1603
3  제주 성산일출봉  33.4590  126.9393
4    경주 불국사  35.7901  129.3316
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   관광지명    5 non-null      object 
 1   위도      5 non-null      float64
 2   경도      5 non-null      float64
dtypes: float64(2), object(1)
memory usage: 248.0+ bytes
None


### 라이브러리 설치

In [72]:
!pip install openrouteservice
!pip install folium
!pip install pandas
!pip install xlrd

### 엑셀 파일을 사용하여 OpenRouteService API를 통해 경로 계산 및 지도 시각화

이 코드는 엑셀 파일에 포함된 관광지 정보를 사용하여 OpenRouteService API를 통해 각 관광지 간의 최적의 경로를 계산하고, 총 거리와 소요 시간을 계산합니다. 또한, 결과를 데이터프레임으로 정리하여 출력합니다.

### 주요 단계:

1. **파일 업로드**:
   - Google Colab의 `files.upload()` 함수를 사용하여 엑셀 파일을 업로드합니다.
   - 업로드된 파일을 `pandas`를 사용하여 데이터프레임으로 로드합니다.

2. **OpenRouteService API 설정**:
   - `openrouteservice.Client`를 사용하여 OpenRouteService API 클라이언트를 설정합니다.

3. **경로 계산 함수**:
   - `calculate_distance_duration` 함수는 주어진 출발지와 도착지 간의 경로를 계산하고, 거리와 소요 시간을 반환합니다.
   - `profile` 매개변수를 사용하여 경로 계산 모드를 지정합니다.

4. **경로 계산 및 결과 저장**:
   - 각 지점 간의 경로를 계산하고, 최적의 프로필을 선택하여 거리와 소요 시간을 계산합니다.
   - 계산된 경로를 리스트에 저장하고, 최종적으로 데이터프레임으로 변환합니다.

5. **결과 출력**:
   - 총 거리와 소요 시간을 출력합니다.
   - 경로 데이터프레임을 출력합니다.

`YOUR_OPENROUTESERVICE_API_KEY` 부분에 실제 OpenRouteService API 키를 입력하여 사용하십시오. 이 코드를 실행하면 엑셀 파일에 포함된 각 관광지 간의 경로를 계산하고, 결과를 데이터프레임으로 출력합니다.

In [75]:
import pandas as pd
import openrouteservice
from google.colab import files
from datetime import datetime

# 파일 업로드
uploaded = files.upload()

# 업로드된 엑셀 파일 로드
df = pd.read_excel(next(iter(uploaded)))

# OpenRouteService API 설정
client = openrouteservice.Client(key='5b3ce3597851110001cf6248dd608457dc56417aa88571512300427c')

# 각 지점 연결하여 거리와 예상 소요시간 계산
routes = []
total_distance = 0
total_duration = 0

def calculate_distance_duration(client, origin, destination, profile):
    try:
        coords = ((origin[1], origin[0]), (destination[1], destination[0]))  # 경도, 위도로 입력
        directions = client.directions(coords, profile=profile)

        if directions:
            distance = directions['routes'][0]['summary']['distance'] / 1000  # meters to kilometers
            duration = directions['routes'][0]['summary']['duration'] / 60  # seconds to minutes
            return distance, duration
        else:
            print(f"No route found from {origin} to {destination} using profile {profile}")
            return "N/A", "N/A"
    except Exception as e:
        print(f"Error calculating route from {origin} to {destination} using profile {profile}: {e}")
        return "Error", "Error"

profiles = ['driving-car', 'foot-walking', 'cycling-regular']

for i in range(len(df) - 1):
    origin = (df.loc[i, '위도'], df.loc[i, '경도'])
    destination = (df.loc[i+1, '위도'], df.loc[i+1, '경도'])

    best_distance = float('inf')
    best_duration = float('inf')
    best_profile = None

    for profile in profiles:
        distance, duration = calculate_distance_duration(client, origin, destination, profile)

        if isinstance(distance, float) and distance < best_distance:
            best_distance = distance
            best_duration = duration
            best_profile = profile

    if best_profile:
        total_distance += best_distance
        total_duration += best_duration
        routes.append({
            '출발지': df.loc[i, '관광지명'],
            '도착지': df.loc[i+1, '관광지명'],
            '거리': f"{best_distance:.2f} km",
            '교통수단': f"{best_duration:.2f} min",
            '프로필': best_profile
        })
    else:
        routes.append({
            '출발지': df.loc[i, '관광지명'],
            '도착지': df.loc[i+1, '관광지명'],
            '거리': "N/A",
            '소요시간': "N/A",
            '교통수단': "N/A"
        })

routes_df = pd.DataFrame(routes)

# 전체 거리와 소요 시간 출력
total_distance_km = f"{total_distance:.2f} km"
total_duration_min = f"{total_duration:.2f} min"

print("총 거리:", total_distance_km)
print("총 소요시간:", total_duration_min)

# 경로 데이터프레임 출력
print(routes_df)

Saving tourist_spots.xlsx to tourist_spots (8).xlsx
Error calculating route from (35.1587, 129.1603) to (33.459, 126.9393) using profile driving-car: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 1: 126.9393000 33.4590000.'}, 'info': {'engine': {'build_date': '2024-05-14T10:47:52Z', 'version': '8.0.1'}, 'timestamp': 1717669457982}})
Error calculating route from (33.459, 126.9393) to (35.7901, 129.3316) using profile driving-car: 404 ({'error': {'code': 2010, 'message': 'Could not find routable point within a radius of 350.0 meters of specified coordinate 0: 126.9393000 33.4590000.'}, 'info': {'engine': {'build_date': '2024-05-14T10:47:52Z', 'version': '8.0.1'}, 'timestamp': 1717669458430}})
총 거리: 1335.92 km
총 소요시간: 3233.93 min
        출발지       도착지         거리         교통수단              프로필
0       경복궁      남산타워    4.16 km    49.93 min     foot-walking
1      남산타워   해운대해수욕장  404.37 km   277.19 min      driv

### 코드 설명

1. **파일 업로드**:
   - Google Colab의 `files.upload()` 함수를 사용하여 엑셀 파일을 업로드합니다.
   - 업로드된 파일을 `pandas`를 사용하여 데이터프레임으로 로드합니다.

2. **OpenRouteService API 설정**:
   - `openrouteservice.Client`를 사용하여 OpenRouteService API 클라이언트를 설정합니다.

3. **지도 생성**:
   - `folium.Map`을 사용하여 지도를 생성합니다. 초기 위치와 줌 레벨을 설정합니다.

4. **경로 계산 함수**:
   - `calculate_distance_duration` 함수는 주어진 출발지와 도착지 간의 경로를 계산하고, 거리와 소요 시간 및 경로 좌표를 반환합니다.
   - `profile` 매개변수를 사용하여 경로 계산 모드를 지정합니다.
   - `radiuses` 매개변수를 사용하여 반경을 1000미터로 설정하여 도로 네트워크를 더 넓게 검색합니다.

5. **경로 계산 및 지도에 추가**:
   - 각 지점 간의 경로를 계산하고, 최적의 프로필을 선택하여 경로를 지도에 추가합니다.
   - 출발지와 도착지에 마커를 추가합니다.

6. **결과 저장 및 출력**:
   - 계산된 경로를 리스트에 저장하고, 전체 거리와 소요 시간을 계산하여 출력합니다.
   - 지도를 HTML 파일로 저장하고 다운로드 링크를 제공합니다.

`YOUR_OPENROUTESERVICE_API_KEY` 부분에 실제 OpenRouteService API 키를 입력하여 사용하십시오. 이 코드를 실행하면 경로를 계산하고, 이를 지도에 표시하고, 다운로드할 수 있는 링크를 제공합니다.

In [71]:
import pandas as pd
import openrouteservice
import folium
from google.colab import files
from datetime import datetime

# 파일 업로드
uploaded = files.upload()

# 업로드된 엑셀 파일 로드
df = pd.read_excel(next(iter(uploaded)))

# OpenRouteService API 설정
client = openrouteservice.Client(key='5b3ce3597851110001cf6248dd608457dc56417aa88571512300427c')

# 지도 생성
m = folium.Map(location=[36.5, 127.5], zoom_start=6)

# 각 지점 연결하여 거리와 예상 소요시간 계산 및 지도에 추가
routes = []
total_distance = 0
total_duration = 0

def calculate_distance_duration(client, origin, destination, profile):
    try:
        coords = ((origin[1], origin[0]), (destination[1], destination[0]))  # 경도, 위도로 입력
        directions = client.directions(coords, profile=profile, radiuses=[1000, 1000])

        if directions:
            distance = directions['routes'][0]['summary']['distance'] / 1000  # meters to kilometers
            duration = directions['routes'][0]['summary']['duration'] / 60  # seconds to minutes
            geometry = directions['routes'][0]['geometry']
            decoded = openrouteservice.convert.decode_polyline(geometry)
            return distance, duration, decoded['coordinates']
        else:
            print(f"No route found from {origin} to {destination} using profile {profile}")
            return "N/A", "N/A", []
    except Exception as e:
        print(f"Error calculating route from {origin} to {destination} using profile {profile}: {e}")
        return "Error", "Error", []

profiles = ['driving-car', 'foot-walking', 'cycling-regular']

for i in range(len(df) - 1):
    origin = (df.loc[i, '위도'], df.loc[i, '경도'])
    destination = (df.loc[i+1, '위도'], df.loc[i+1, '경도'])

    best_distance = float('inf')
    best_duration = float('inf')
    best_profile = None
    best_coords = []

    for profile in profiles:
        distance, duration, coords = calculate_distance_duration(client, origin, destination, profile)

        if isinstance(distance, float) and distance < best_distance:
            best_distance = distance
            best_duration = duration
            best_profile = profile
            best_coords = coords

    if best_profile:
        total_distance += best_distance
        total_duration += best_duration

        # 경로 추가
        folium.PolyLine(locations=[(coord[1], coord[0]) for coord in best_coords], color='blue', weight=5).add_to(m)

        # 출발지 및 도착지 마커 추가
        folium.Marker(location=[origin[0], origin[1]], popup=df.loc[i, '관광지명']).add_to(m)
        folium.Marker(location=[destination[0], destination[1]], popup=df.loc[i+1, '관광지명']).add_to(m)

        routes.append({
            '출발지': df.loc[i, '관광지명'],
            '도착지': df.loc[i+1, '관광지명'],
            '거리': f"{best_distance:.2f} km",
            '소요시간': f"{best_duration:.2f} min",
            '교통수단': best_profile
        })
    else:
        routes.append({
            '출발지': df.loc[i, '관광지명'],
            '도착지': df.loc[i+1, '관광지명'],
            '거리': "N/A",
            '소요시간': "N/A",
            '교통수단': "N/A"
        })

routes_df = pd.DataFrame(routes)

# 전체 거리와 소요 시간 출력
total_distance_km = f"{total_distance:.2f} km"
total_duration_min = f"{total_duration:.2f} min"

print("총 거리:", total_distance_km)
print("총 소요시간:", total_duration_min)

# 경로 데이터프레임 출력
print(routes_df)

# 지도 저장 및 출력
map_path = 'map.html'
m.save(map_path)
print(f"Map saved to {map_path}")

# 파일 다운로드 링크 제공
files.download(map_path)


Saving tourist_spots.xlsx to tourist_spots (6).xlsx
총 거리: 1268.31 km
총 소요시간: 1178.39 min
        출발지       도착지         거리        소요시간          교통수단
0       경복궁      남산타워    4.16 km   49.93 min  foot-walking
1      남산타워   해운대해수욕장  404.37 km  277.19 min   driving-car
2   해운대해수욕장  제주 성산일출봉  400.09 km  404.30 min   driving-car
3  제주 성산일출봉    경주 불국사  459.69 km  446.98 min   driving-car
Map saved to map.html


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>